In [ ]:
import os
os.environ['PROJ_LIB'] = '/usr/local/anaconda3/share/proj'
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import pandas as pd
import numpy as np
from matplotlib import colors
import glob
import math
from scipy.optimize import curve_fit
from pylab import flipud
import seaborn as sns
sns.set ( )
import random as rnd

In [ ]:
def distance ( _lat1, _lon1, _lat2, _lon2 ):
    _R = 6377.083
    
    _dLat = math.radians ( _lat2 - _lat1 )
    _dLon = math.radians ( _lon2 - _lon1 )
    _lat1 = math.radians ( _lat1 )
    _lat2 = math.radians ( _lat2 )
 
    _a = math.sin ( _dLat / 2.0 ) ** 2.0 + math.cos ( _lat1 ) * math.cos ( _lat2 ) * math.sin ( _dLon / 2.0 ) ** 2.0
    _c = 2 * math.asin ( math.sqrt ( _a ) )
 
    return ( abs ( _R * _c ) )

In [ ]:
def influence_decay ( _distance, _influenceradius ):
    _influence = np.exp ( -1.0 * ( _distance / _influenceradius ) )
    return ( _influence )

In [ ]:
def influence_map ( _data, _size, _show_prog = False ):
    llcrnrlat = 5.5
    urcrnrlat = 20.5
    llcrnrlon = 115
    urcrnrlon = 130
    _z = np.zeros ( ( _size, _size ) )
    _x = np.linspace ( llcrnrlon, urcrnrlon, _z.shape [ 1 ] )
    _y = np.linspace ( llcrnrlat, urcrnrlat, _z.shape [ 0 ] )
    _xx, _yy = np.meshgrid ( _x, _y )
    _station_lat = _data [ 'Latitude' ].tolist ( )
    _station_lon = _data [ 'Longitude' ].tolist ( )
    _station_reliability = _data [ 'Reliability' ].tolist ( )
    _station_influence = _data [ 'a0' ].tolist ( )
    for i in range ( _z.shape [ 1 ] ):
        if ( _show_prog ):
            print ( i )
        for j in range ( _z.shape [ 0 ] ):
            _ref_lat = _y [ j ]
            _ref_lon = _x [ i ]
            _ref_reliability = [ ]
            for k in range ( len ( _data ) ):
                _distance = distance ( _ref_lat, _ref_lon, _station_lat [ k ], _station_lon [ k ] )
                _ref_reliability.append ( _station_reliability [ k ] * influence_decay ( _distance, _station_influence [ k ] ) )
            _z [ j ] [ i ] = max ( _ref_reliability )
    return ( _xx, _yy, _z )

In [ ]:
year = 2017
estsize = 350
data = pd.read_csv ( str ( year ) + 'StationInfluence.csv' )
xx, yy, z = influence_map ( data, estsize )
np.save ( str ( year ) + 'Attack100/' + str ( year ) + '-' + str ( estsize ) + '-x.npy', xx )
np.save ( str ( year ) + 'Attack100/' + str ( year ) + '-' + str ( estsize ) + '-y.npy', yy )
np.save ( str ( year ) + 'Attack100/' + str ( year ) + '-' + str ( estsize ) + '-maxz.npy', z )
for _iijj in range ( 1, 20 ):
    percent = 1.00 - ( 0.05 * _iijj )
    removed_len = int ( len ( data ) * ( 1 - percent ) )
    for _trials in range ( 10 ):
        removed_index = rnd.sample ( range ( 0, len ( data ) ), removed_len)
        data_dropped = data.drop ( data.index [ removed_index ] )
        xx, yy, z = influence_map ( data_dropped, estsize )
        np.save ( str ( year ) + 'Attack100/' + str ( year ) + '-' + str ( estsize ) + '-' + str ( _iijj ) + '-maxz-' + str ( _trials ) + '.npy', z )
        print ( 'Progress at ' + str ( percent ) + ' at trial number ' + str ( _trials + 1 ) )
    print ( len ( data ) )
print ( 'DONE' )